Code for scraping all the urls of middle-school in the auonomous province of Trento

In [ ]:
import bs4, requests
import pandas as pd

ms_typeEdu = 'middleschool' 
ms_url = 'https://www.tuttitalia.it/trentino-alto-adige/provincia-autonoma-di-trento/50-scuole/scuola-secondaria-di-primo-grado/'
ms_pre_url = 'https://www.tuttitalia.it' 

ms_response = requests.get(ms_url) 
ms_response.raise_for_status()
ms_soup = bs4.BeautifulSoup(ms_response.text, 'html.parser') 

ms_municipalities_urls = [] 

ms_ct_tables = ms_soup.find_all('table', class_='ct')

for ms_ct_table in ms_ct_tables:
    ms_vm_tables = ms_ct_table.find_all('table', class_='vm')

    for ms_vm_table in ms_vm_tables:
        ms_zp_tds = ms_vm_table.find_all('td', class_='zp')

        for ms_zp_td in ms_zp_tds: 
            ms_a_muns = ms_zp_td.find('a') 
            ms_href = ms_a_muns.get('href')
            ms_mun_url = ms_pre_url + ms_href 
            
            ms_municipalities_urls.append(ms_mun_url)

ms_df_urls = pd.DataFrame({'url': ms_municipalities_urls, 'typeEdu': ms_typeEdu})
print(ms_df_urls) 


Extracting all the information about all the middle schools reading the df_urls

In [ ]:
import bs4, requests
import pandas as pd
import re 

file_path = '../../Raw data/edu/middle_school.csv'
ms_school_infos = []

for ms_index, ms_row in ms_df_urls.iterrows():
    ms_url = ms_row['url']
    ms_typeEdu = ms_row['typeEdu'] 

    ms_response = requests.get(ms_url) 
    ms_response.raise_for_status()
    ms_soup = bs4.BeautifulSoup(ms_response.text, 'html.parser') 

    ms_div_type = ms_soup.find('div', class_='aj')
    ms_divs_if = ms_div_type.find_all('div', class_='if')

    for ms_div_if in ms_divs_if:
        ms_id = ms_div_if.get('id')
        ms_div_kk = ms_div_if.find('div', class_='kk') 
        
        if ms_div_kk:
            ms_name = ms_div_kk.find('h3').text
            
            ms_text_elements = ms_div_kk.find_all(string=True)
            ms_address = ' '.join(ms_text_elements).strip()
            ms_address_match = re.search(r'(Via|Corso|Piazza|Strada|Loc).*?TN', ms_address)
            if ms_address_match:
                ms_address = ms_address_match.group()
    
            school_info = [ms_id, ms_typeEdu, ms_name, ms_address]
            ms_school_infos.append(school_info)

ms_df = pd.DataFrame(ms_school_infos, columns=['id', 'type', 'name', 'address'])
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)
ms_df.to_csv(file_path, index=False)
#print(ms_df)  